In [1]:
import torch

/mnt/cfs/anaconda3/envs/bojun_pykt/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pykt.utils.wandb_utils import WandbUtils
import pandas as pd
import os
import yaml

# small to large

In [3]:
wandb_key = "fa9e12af1b9afc9a0dc55285128aec2f185d0e6c"
os.environ['WANDB_API_KEY'] = wandb_key
wandb_api = WandbUtils(user='1540712456', project_name=f'simplekt_project')

self.sweep_dict is {'bridge2algebra2006_simplekt_qid_prediction': '9egmxh27', 'peiyou_simplekt_qid_prediction': 'v5n9800i', 'algebra2005_simplekt_qid_prediction': '1kjhlrri', 'nips_task34_simplekt_qid_prediction': 'odmwid9e', 'assist2009_simplekt_qid_prediction': 'eay390qv', 'peiyou_simplekt_qid_0': 'hyha1ql9', 'bridge2algebra2006_simplekt_qid_0': 'whagcrrf', 'algebra2005_simplekt_qid_0': 'teomtc09', 'nips_task34_simplekt_qid_0': 'kvszqpjy', 'assist2009_simplekt_qid_0': '22tmydo0'}


In [4]:
def get_runs_result(runs):
    result_list = []
    for run in runs:
        result = {}
        result.update(run.summary._json_dict)
        model_config = {k: v for k, v in run.config.items()
                        if not k.startswith('_') and type(v) not in [list, dict]}
        result.update(model_config)
        result['name'] = run.name
        result['path_id'] = run.path[-1]
        result['state'] = run.state
        result_list.append(result)
    runs_df = pd.DataFrame(result_list)
    runs_df['create_time'] = runs_df['_timestamp']
    model_config_keys = list(model_config.keys())
    return runs_df,model_config_keys

In [5]:
#修改wandb配置文件
def generate_wandb(dataset_name, model_name, emb_type, fpath, ftarget, model_path):
    with open(fpath,"r") as fin,\
        open(ftarget,"w") as fout:
        data = yaml.load(fin, Loader=yaml.FullLoader)
        name = ftarget.split('_')
        data['name'] = '_'.join([dataset_name, model_name, emb_type, 'prediction'])
        data['parameters']['save_dir']['values'] = model_path
        data['parameters']['dataset_name']['values'] = [dataset_name]
        yaml.dump(data, fout)

In [6]:
def generate_sweep(wandb_key, pred_dir, fallsh, ftarget, generate_all=False):
    # with open(wandb_path) as fin:
    #     wandb_config = json.load(fin)
    pre = "WANDB_API_KEY=" + wandb_key + " wandb sweep "
    if generate_all:
        files = os.listdir(pred_dir)
        files = sorted(files)
        for f in files:
            fpath = os.path.join(pred_dir, f)
            fallsh.write(pre + fpath + " -p {}".format(project_name) + "\n")
    else:
        fallsh.write(pre + ftarget + " -p {}".format(project_name) + "\n")

In [10]:
user, project_name = "1540712456", "simplekt_project"
dataset_names, model_names, emb_types, pred_type, fold = "peiyou", "simplekt", "qid", "pretrain", "0"
fpath = "./seedwandb/predict.yaml"

with open("start_predict.sh","w") as fallsh:
    for dataset_name in dataset_names.split(","):
        for model_name in model_names.split(","):
            for emb_type in emb_types.split(","):
                # dconfig = dict()
                sweep_name = "_".join([dataset_name, model_name, emb_type, fold])
                sweep_id = wandb_api.sweep_dict[sweep_name]
                sweep = wandb_api.api.sweep(f"{user}/{project_name}/{sweep_id}")
                df, model_config_keys = get_runs_result(sweep.runs)
                new_df = df[df['model_save_path'].notna()]
                # 对new_df按照从小到大的validauc排序，
                new_df = new_df.sort_values(by='validauc', ascending=True)
                new_df = new_df.head(200)
                if new_df.shape != df.shape:
                    print("df.shape not correct, maybe sth error!!!")
                model_path = new_df["model_save_path"].tolist()
                model_path = ["/".join(x.split("/")[:-1]) for x in model_path] 
                ftarget = ftarget = os.path.join("pred_wandbs", "{}_{}_{}_predict.yaml".format(dataset_name, model_name, emb_type))
                generate_wandb(dataset_name, model_name, emb_type, fpath, ftarget, model_path)
                # dconfig["model_path_fold_first"] = model_path
                generate_sweep(wandb_key, "pred_wandbs", fallsh, ftarget)

df.shape not correct, maybe sth error!!!


In [8]:
new_df

,best_epoch,model_save_path,_step,_wandb,_runtime,validacc,validauc,_timestamp,d_ff,fold,...,train_ratio,dataset_name,final_fc_dim,final_fc_dim2,learning_rate,num_attn_heads,name,path_id,state,create_time
224,200.0,models/simplekt_tiaocan_algebra2005/algebra200...,0.0,{'runtime': 387},389.062384,0.702431,0.596257,1.704804e+09,64,0,...,1,algebra2005,64,64,0.00001,8,dutiful-sweep-8,tj9fat62,finished,1.704804e+09
228,199.0,models/simplekt_tiaocan_algebra2005/algebra200...,0.0,{'runtime': 304},305.881454,0.702443,0.661335,1.704804e+09,256,0,...,1,algebra2005,256,256,0.00001,4,denim-sweep-6,uprlucbw,finished,1.704804e+09
231,198.0,models/simplekt_tiaocan_algebra2005/algebra200...,0.0,{'runtime': 321},321.556174,0.702443,0.665602,1.704804e+09,64,0,...,1,algebra2005,256,64,0.00001,8,tough-sweep-1,vi6td0oi,finished,1.704804e+09
220,199.0,models/simplekt_tiaocan_algebra2005/algebra200...,0.0,{'runtime': 412},414.514103,0.702443,0.675470,1.704804e+09,256,0,...,1,algebra2005,256,256,0.00001,8,bright-sweep-12,cvd99vlz,finished,1.704804e+09
230,200.0,models/simplekt_tiaocan_algebra2005/algebra200...,0.0,{'runtime': 351},354.698970,0.703567,0.746540,1.704804e+09,64,0,...,1,algebra2005,64,64,0.00001,4,unique-sweep-2,dpjdesgz,finished,1.704804e+09
51,197.0,models/simplekt_tiaocan_algebra2005/algebra200...,0.0,{'runtime': 374},380.456258,0.713273,0.749002,1.704810e+09,256,0,...,1,algebra2005,256,256,0.00001,8,good-sweep-184,wb3qqklp,finished,1.704810e+09
226,200.0,models/simplekt_tiaocan_algebra2005/algebra200...,0.0,{'runtime': 355},355.331418,0.761082,0.789052,1.704804e+09,64,0,...,1,algebra2005,256,64,0.00010,4,sparkling-sweep-5,6a5a3uqx,finished,1.704804e+09
217,199.0,models/simplekt_tiaocan_algebra2005/algebra200...,0.0,{'runtime': 339},341.444917,0.817113,0.860076,1.704804e+09,64,0,...,1,algebra2005,64,256,0.00010,8,exalted-sweep-15,cn8lkbw2,finished,1.704804e+09
227,197.0,models/simplekt_tiaocan_algebra2005/algebra200...,0.0,{'runtime': 327},325.857526,0.819058,0.864739,1.704804e+09,64,0,...,1,algebra2005,256,256,0.00010,8,smart-sweep-4,9dwgwveb,finished,1.704804e+09
229,198.0,models/simplekt_tiaocan_algebra2005/algebra200...,0.0,{'runtime': 362},365.598064,0.824953,0.872205,1.704804e+09,256,0,...,1,algebra2005,256,64,0.00010,4,chocolate-sweep-3,do00iss0,finished,1.704804e+09


# 4Bto7B

In [17]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("shyann/unikt-4b-finetune_bd2006")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })


In [20]:
runs_df["summary"]["model_save_path"]

KeyError: 'model_save_path'